In [1]:
import pika
import uuid

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

In [3]:
class FibonacciRpcClient(object):

    def __init__(self):
        #self.connection = pika.BlockingConnection(
        #    pika.ConnectionParameters(host='localhost'))
        #self.channel = self.connection.channel()
        

        self.connection = pika.BlockingConnection(parameters)
        self.channel = self.connection.channel()

        result = self.channel.queue_declare('', exclusive=True)
        print(result)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(
            queue=self.callback_queue,
            on_message_callback=self.on_response,
            auto_ack=True)

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4())
        print(self.corr_id)
        self.channel.basic_publish(
            exchange='',
            routing_key='rpc_queue',
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
            ),
            body=str(n))
        while self.response is None:
            self.connection.process_data_events()
        return int(self.response)

In [4]:
fibonacci_rpc = FibonacciRpcClient()

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-dn4TLbG4SwHhBokSaI-JCw'])>"])>


In [5]:
print(" [x] Requesting fib(30)")
response = fibonacci_rpc.call(30)
print(" [.] Got %r" % response)

 [x] Requesting fib(30)
d1ddbc98-3f35-45db-a29e-8ad1d5ee1ae7
 [.] Got 832040


In [6]:
print(" [x] Requesting fib(20)")
response = fibonacci_rpc.call(20)
print(" [.] Got %r" % response)

 [x] Requesting fib(20)
a1dd7d8b-b3ee-4fde-90cd-e6afdeaeac3e
 [.] Got 6765


In [7]:
print(" [x] Requesting fib(32)")
response = fibonacci_rpc.call(32)
print(" [.] Got %r" % response)

 [x] Requesting fib(32)
05a9364c-a54b-40b3-8b1c-c6dc946d991a
 [.] Got 2178309


In [8]:
print(" [x] Requesting fib(14)")
response = fibonacci_rpc.call(14)
print(" [.] Got %r" % response)

 [x] Requesting fib(14)
b9ba985e-c3c5-4be5-9f92-5574872733d8
 [.] Got 377
